<img src=../Img/pencil-2841771.png style="width:39%">
<p style="background-color:pink; font-size:125%; color: black;text-align:center">Домашнее задание</p>
<p>Выполните домашнее задание с помощью Jupyter Notebook.</p>

1. Студентам, обучающимся в ВГУ, присвойте порядковый номер, упорядочив по фамилии и имени.

2. Предметы, по которым сдавались экзамены, ранжируйте по среднему баллу, полученному на экзаменах. Результат должен содержать название предмета, средний балл, номер ранга без пропусков. Предметам с большим значением среднего балла присвойте меньшее значение ранга. Значение среднего балла округлите до двух десятичных знаков.

3. Ранжируйте преподавателей (без пропуска значений ранга) по количеству преподаваемых ими предметов.

4. Ранжируйте семестры по количеству учебных часов.

5. Рассчитайте относительный прирост суммарного количества учебных часов предметов семестра по мере увеличения номера семестра.

6. Для каждого студента, обучающегося в своём городе, рассчитайте разницу в возрасте в днях с самым старшим из всех студентов, обучающихся в своём городе. Результат упорядочить по разности в возрасте.

In [18]:
from sqlalchemy import create_engine
import pandas as pd

In [19]:
engine = create_engine('sqlite+pysqlite:///../Data/Students.db')

In [20]:
conn = engine.connect()

In [21]:
# 1. Студентам, обучающимся в ВГУ, присвойте порядковый номер, упорядочив по фамилии и имени.

s = """
SELECT rank () OVER (ORDER BY st.surname, st.name) AS dense_rank,
st.student_id, st.surname, st.name
FROM student st JOIN university u ON st.univ_id=u.univ_id
WHERE u.univ_name = 'ВГУ'
"""

pd.read_sql(s, conn)

,dense_rank,student_id,surname,name
0,1,78,Агафонов,Василий
1,2,113,Аксенов,Валерий
2,3,55,Белкин,Вадим
3,4,99,Борисова,Марина
4,5,88,Жданова,Ирина
5,6,12,Зайцева,Ольга
6,7,1,Иванов,Иван
7,8,10,Кузнецов,Борис
8,9,654,Лукин,Артем
9,10,76,Мельникова,Дарья


In [22]:
# 2. Предметы, по которым сдавались экзамены, ранжируйте по среднему баллу, полученному на экзаменах. Результат должен содержать название предмета, средний балл, номер ранга без пропусков. Предметам с большим значением среднего балла присвойте меньшее значение ранга. Значение среднего балла округлите до двух десятичных знаков.

s = """
WITH A as (
SELECT su.subj_id, su.subj_name, avg(em.mark) AS avg_m
from exam_marks em Join subject su on em.subj_id=su.subj_id
GROUP by em.subj_id
)

SELECT dense_rank() OVER(ORDER by A.avg_m DESC) as dense_rank, A.subj_id, A.subj_name, round(A.avg_m, 2) AS avd_mark
FROM A
"""

pd.read_sql(s, conn)

,dense_rank,subj_id,subj_name,avd_mark
0,1,14,Обработка данных и моделирование в Excel,4.00
1,2,94,Английский язык,3.75
2,3,10,Информатика,3.33
3,3,11,Программирование на языках Python и SQL,3.33
4,4,56,История,3.14
5,5,73,Физкультура,3.00
6,6,22,Физика,2.83
7,7,13,ОБЖ,2.50
8,8,12,Анализ данных,2.00
9,8,43,Математика,2.00


In [23]:
# 3. Ранжируйте преподавателей (без пропуска значений ранга) по количеству преподаваемых ими предметов.

s = """
WITH A as (
SELECT lu.lecturer_id, lu.surname, lu.name, count(sl.subj_id) AS count_subj
from lecturer lu Join subj_lect sl on lu.lecturer_id=sl.lecturer_id
GROUP by lu.lecturer_id
)

SELECT dense_rank() OVER (ORDER BY A.count_subj desc) as ranc, A.*
FROM A;
"""

pd.read_sql(s, conn)

,ranc,lecturer_id,surname,name,count_subj
0,1,2,Зайцева,Юлия,2
1,1,6,Максимова,Иванна,2
2,1,11,Куликова,Ираида,2
3,1,16,Волков,Михаил,2
4,1,22,Беспалов,Юлиан,2
5,1,108,Струков,Николай,2
6,2,1,Сафонова,Лукия,1
7,2,3,Мамонтова,Акулина,1
8,2,5,Брагина,Нинель,1
9,2,12,Кудрявцева,Анна,1


In [24]:
# 4. Ранжируйте семестры по количеству учебных часов.

s = """SELECT semester, SUM(hour) AS "Кол-во учебных часов", rank() over (order by sum(hour)) AS "Ранг по кол-ву учебных часов" FROM subject GROUP BY semester"""

pd.read_sql(s, conn)

,semester,Кол-во учебных часов,Ранг по кол-ву учебных часов
0,5,34,1
1,3,92,2
2,2,124,3
3,1,132,4
4,4,138,5


In [25]:
# 5. Рассчитайте относительный прирост суммарного количества учебных часов предметов семестра по мере увеличения номера семестра.

s = """
SELECT semester, SUM(hour) AS "Кол-во учебных часов",
(sum(hour) - lag(sum(hour)) over ()) AS "Прирост суммарного кол-ва учебных часов"
FROM subject GROUP BY semester ORDER BY semester
"""

pd.read_sql(s, conn)

,semester,Кол-во учебных часов,Прирост суммарного кол-ва учебных часов
0,1,132,NaN
1,2,124,-8.0
2,3,92,-32.0
3,4,138,46.0
4,5,34,-104.0


In [26]:
# 6. Для каждого студента, обучающегося в своём городе, рассчитайте разницу в возрасте в днях с самым старшим из всех студентов, обучающихся в своём городе. Результат упорядочить по разности в возрасте.

s = """
SELECT student_id, name, surname, birthday,
       (julianday(birthday) -
        (SELECT min(julianday(birthday))
         FROM student JOIN university u on u.univ_id = student.univ_id
         WHERE student.city = u.city)) age_day_diff FROM student
             JOIN university u on u.univ_id = student.univ_id
                                                    WHERE student.city = u.city ORDER BY age_day_diff
"""

pd.read_sql(s, conn)

,student_id,name,surname,birthday,age_day_diff
0,6,Вадим,Сидоров,1999-06-07 00:00:00.000000,0.0
1,55,Вадим,Белкин,2000-01-07 00:00:00.000000,214.0
2,265,Андрей,Павлов,2000-11-05 00:00:00.000000,517.0
3,32,Павел,Котов,2001-02-28 00:00:00.000000,632.0
4,203,Екатерина,Петрова,2001-05-21 00:00:00.000000,714.0
5,200,Александр,Баранов,2001-08-18 00:00:00.000000,803.0
6,654,Артем,Лукин,2001-12-01 00:00:00.000000,908.0
7,207,Федор,Миронова,2002-04-15 00:00:00.000000,1043.0
8,58,Вера,Журавлева,2003-06-14 00:00:00.000000,1468.0
